In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd 
from sklearn import preprocessing
import os
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import hyperopt
print(hyperopt.__version__)
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
PATH_DATASET = r'Dataset.xlsx'
Target_Year = r'T22'
PREDICTION_DATASET=r'PR_Dataset.csv'

In [ ]:
df = pd.read_excel(PATH_DATASET)
Features=['LST','NDBI','NDVI','BD','RD','X','Y','SRTM','slope','MNDWI']
X = df[Features]
print(X.columns)
y = df[[Target_Year]]
y = y.to_numpy()
scaler = MinMaxScaler()
Xs = scaler.fit_transform(X)
print(Xs.shape)
print(y.shape)

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

i = 0
for train_index, test_index in cv.split(X):
  print("loop:", i)
  if i == 2:
    print( "TRAIN:", train_index, "TEST:", test_index)
    break
  else:
    i+=1
Xtrain = Xs[train_index]
print(np.shape(Xtrain))
ytrain = y[train_index]

Xtest = Xs[test_index]
print(np.shape(Xtest))
ytest = y[test_index]



param_hyperopt= {
    # learning rate
    'eta': hp.loguniform('eta', np.log(0.01), np.log(1)),
    'max_depth': scope.int(hp.quniform('max_depth', 5, 35, 1)),
    'n_estimators' : scope.int(hp.quniform('n_estimators', 50, 250, 10)),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 10.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0)
    }

print(f'Shape of Train {Xtrain.shape}')
print(f'Shape of Test {Xtest.shape}')

In [ ]:
def hyperopt(param_space, X_train, y_train, num_eval):
    
    start = time.time()
    
    def objective_function(params):
        reg = XGBRegressor(**params)
        score = cross_val_score(reg, Xtrain, ytrain, cv=5).mean()
        return {'loss': -score, 'status': STATUS_OK}

    trials = Trials()
    best_param = fmin(objective_function, 
                      param_space, 
                      algo=tpe.suggest, 
                      max_evals=num_eval, 
                      trials=trials,
                      rstate= np.random.RandomState(1))
    loss = [x['result']['loss'] for x in trials.trials]
    

   
    
    reg_best = XGBRegressor(
                            eta=best_param['eta'],
                            max_depth=int(best_param['max_depth']),
                            n_estimators=int(best_param['n_estimators']),
                            min_child_weight=best_param['min_child_weight'],
                            colsample_bytree=best_param['colsample_bytree'],
                            subsample=best_param['subsample'],
                            gamma=best_param['gamma'],
                            reg_lambda=best_param['reg_lambda']
                            )
    reg_best.fit(Xtrain, ytrain)
    
    print("")
    print("##### Results")
    print("Score best parameters: ", min(loss)*-1)
    print("Best parameters: ", best_param)
    print("Time elapsed: ", time.time() - start)
    print("Parameter combinations evaluated: ", num_eval)
    
    return trials, reg_best

In [ ]:
results_hyperopt, reg_best = hyperopt(param_hyperopt, Xtrain, ytrain, 20)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams['font.size'] = 12 
losses = [x['result']['loss'] for x in results_hyperopt.trials]
best_so_far = np.maximum.accumulate(-np.array(losses))
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(losses) + 1), best_so_far, marker='o', markersize=4, color='green') # مارکرها را کمی کوچکتر کردم
plt.title('Hyperopt Convergence Plot')
plt.xlabel('Iteration')
plt.ylabel('Best CV Score')
plt.grid(True)
try:
    plt.savefig(r'convergence_plot.png', dpi=400, bbox_inches='tight')
except Exception as e:
    print(f"{e}")
plt.show()

In [ ]:
dftest = df.loc[test_index]
# Prediction
dftest["y_pred"] = reg_best.predict(Xtest)
dftest.to_excel(r'TestDS.xlsx', index=False)
dftest.head()

In [ ]:
print ("RMSE: ", np.round(mean_squared_error(dftest["y_pred"], dftest["t21"], squared=False),10))
print ("MAE: ", np.round(mean_absolute_error(dftest["y_pred"], dftest["t21"]),4))
print ("BIAS: ", np.round(np.mean(dftest["y_pred"]- dftest["t21"]),4))
from scipy.stats import pearsonr
corr, _ = pearsonr(dftest["y_pred"], dftest["t21"])
print('Pearsons R2 correlation: %.4f' % corr**2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
from scipy.stats import gaussian_kde, pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error
x, y = dftest["y_pred"].to_numpy(), dftest["t21"].to_numpy()
rmse = mean_squared_error(y, x, squared=False)
mae = mean_absolute_error(y, x)
bias = np.mean(x - y)
corr, _ = pearsonr(x, y)
r2 = corr**2
stats_text = (
    f"$R^2$    : {r2:6.2f}\n"
    f"RMSE (°C): {rmse:6.2f}\n"
    f"MAE  (°C): {mae:6.2f}\n"
    f"BIAS (°C): {bias:6.2f}"
)

fig, ax = plt.subplots(figsize=(6,6), dpi=300)
plt.rc('font', family='serif')
sb.regplot(x=x, y=y, ax=ax, line_kws={"color": "red"}, scatter=False)
xy = np.vstack([x, y])
z = gaussian_kde(xy)(xy)
sc = ax.scatter(x, y, c=z, s=8, cmap='jet')
ax.set_xlabel("Measured T2 (°C)", fontsize=14)
ax.set_ylabel("Predicted T2 (°C)", fontsize=14)
ax.text(0.05, 0.95, stats_text, transform=ax.transAxes,
        fontsize=12, verticalalignment='top', family='monospace',
        bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.7))
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label("Point Density", fontsize=12)
plt.tight_layout()
plt.savefig(r"FeatureImportance.png",
            dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
def C_FI(Model):
    # Get feature importances
    feature_importance = Model.feature_importances_

    # Calculate total feature importance
    total_importance = sum(feature_importance)

    # Normalize feature importance to percentages
    feature_importance_percentage = (feature_importance / total_importance) * 100

    # Create a DataFrame with feature names and their corresponding importance scores
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance_percentage})

    # Sort the DataFrame by importance scores in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
    feature_importance_df.to_csv(r'FeatureImportance.csv', index=False)
    print('Feature Importance File is generated!!')
C_FI(reg_best)

In [ ]:
import pickle
with open(r'Train_Model.pkl','wb') as f:
    pickle.dump(reg_best,f)
    print("Model Saved!")    

In [ ]:
df_Pred=pd.read_csv(r'Pred_DS.csv')
X_Pred= df_Pred[Features]
Xs_Pred=scaler.fit_transform(X_Pred)
print(np.shape(Xs_Pred))
df_Pred[Target_Year]=reg_best.predict(Xs_Pred)
df_Pred.to_excel(r'PD.xlsx', index=False)